# Stardist 3D Cell Nuclei Segmentation

This notebook demonstrates 3D cell nuclei segmentation using StarDist, a deep learning model for instance segmentation of star-convex objects. It covers loading a 3D microscopy volume with BioIO, normalizing the data, running StarDist inference to generate instance masks, and visualizing the results overlaid on the raw volume in Napari.

## Learning Outcomes

By the end of this notebook, you will be able to:

1. **Load and inspect a 3D microscopy volume** using BioIO and Dask for memory-aware access.
2. **Normalize and prepare 3D data** for StarDist inference, including axis handling and scaling.
3. **Run a pretrained StarDist 3D model** to obtain instance masks for nuclei.
4. **Evaluate segmentations in Napari** by overlaying label masks on the raw volume and adjusting visualization settings.

## Load the resampled Lund data

We will use [BioIO](https://bioio-devs.github.io/bioio/OVERVIEW.html) to read the 3D `lund1051_resampled.tif` volume. BioIO exposes a Dask-backed array so we can defer loading until needed for visualization or model inference.

In [ ]:
from bioio import BioImage

image_handle = BioImage("../data/lund1051_resampled.tif")
image_data = image_handle.dask_data.squeeze()
image_data

## Visualize the volume in Napari

Napari provides interactive 3D navigation. Rendering the Dask array keeps memory usage manageable until tiles are requested.

In [ ]:
import napari

viewer = napari.Viewer(ndisplay=3)
viewer.add_image(image_data, name="Lund resampled", rendering="attenuated_mip")
napari.run()

## Prepare and run StarDist 3D

StarDist models predict object centers and star-convex polygons/surfaces. We will use a pretrained 3D model and normalize the volume intensities before inference.

In [ ]:
from stardist.models import StarDist3D
from csbdeep.utils import normalize

# Load pretrained 3D model (ships with StarDist)
model = StarDist3D.from_pretrained("3D_demo")
model

In [ ]:
help(model.predict_instances)

In [ ]:
# Materialize the Dask array for inference and normalize intensities
volume = image_data.compute()
volume_norm = normalize(volume, 1, 99)

labels, details = model.predict_instances(
    volume_norm,
    prob_thresh=0.5,
    nms_thresh=0.3,
    )

## Inspect StarDist results in Napari

Overlay labels on the raw data and adjust opacity/colormap to spot under- or over-segmentation.

In [ ]:
viewer = napari.Viewer(ndisplay=3)
viewer.add_image(volume_norm, name="Lund resampled (norm)", rendering="attenuated_mip")
viewer.add_labels(labels, name="StarDist nuclei", opacity=0.5)
napari.run()

## Practice questions

1. How does changing the normalization percentiles affect small, dim nuclei?
2. Which visualization settings (gamma, colormap, opacity) help you spot missed nuclei?
3. If you see merged objects, which StarDist parameters would you adjust first (e.g., `prob_thresh`, `nms_thresh`)?
4. What happens when you switch to a model fine-tuned on your own annotations?

## Useful Resources

- [Stardist Napari plugin](https://github.com/stardist/stardist-napari)
- [Stardist Notebook Examples](https://github.com/stardist/stardist/tree/main/examples)
- [Notebook fro big images](https://github.com/stardist/stardist/blob/main/examples/other2D/predict_big_data.ipynb)